In [ ]:
import json
import requests
import urllib3
import time
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
data_json = 'data.json'

def get_data(filename=data_json):
  with open(filename) as json_file:
      data = json.load(json_file)
  return data

def set_data(data, filename=data_json):
  data = json.dumps(data, indent=2, ensure_ascii=False)
  with open(filename, 'w') as f:
    f.write(data)

In [ ]:
data = get_data('put_mark.json')
data

In [ ]:
headers = {
    'Cookie': data['cookie'],
    'Sec-Fetch-Dest': 'empty',
    'Content-Type': 'application/x-www-form-urlencoded',
}
body = data['body']

url = 'https://sh-open.ris61edu.ru/actions/web_edu.core.fgos.register.tabs.main.actions.RegisterPack/registerrowsaction'

In [190]:
# string, int, string
def put_mark(lesson_id, student_id, mark):
    
    reqest_form_paramenters = '{"%s":"%s","id":%d}' % (lesson_id, mark, student_id)
    print(reqest_form_paramenters)

    response = requests.post(url, headers=headers, data=body.replace('*params*',reqest_form_paramenters).encode('utf-8') , verify=False).text
    print(response)
    if '"success": true' not in response:
        print("!!! Response is empty - restart !!!")
        time.sleep(1)
        put_mark(lesson_id, student_id, mark)
    

In [ ]:
# lesson_id = "mix_138710669_143456746"
# student_id = 491349
# mark = "5"

# put_mark("mix_138710669_143456746", 491349, "")

In [ ]:
with open("columns.json", "r") as read_file:
    columns = json.load(read_file)
print(columns)

In [ ]:
with open("rows.json", "r") as read_file:
    students = json.load(read_file)
print(len(students))

In [ ]:
def get_mark(student, lesson_id):
    if lesson_id in student:
        return student[lesson_id].split('.')[0]
    return ""

In [ ]:
marks_text = """
 4   5   4 5  4  4     444
   23  3 23    23      333
 23   3  2 5  23  3    333
 23  3   ннннннн 3     333
   3 3   3 55 2 3      333

 34    4 н4   4  4     444
   4 5 4 4 5   3 44    444
 23   3  3    3 3      333
  23   3 3 5   3 3     333
 3   3   3    3   н    333
   4 3   4 55  4       444
 5    5  4 5  4 55     555
 23      н    2  3     333
   3  4  3 5   4  н    444
    3    23   2 3      333

  4    4 3 5  43  4    444
     34  3 55  34      444
   34    4  5 4  4     444
 4    4  4 5   3  4    444
"""


In [ ]:
marks = marks_text.split('\n')
del marks[0]
del marks[len(marks) - 1]

In [ ]:
if len(marks) != len(students):
    raise ValueError(f'Marks error! Students count is wrong')
for lesson_index, student_marks in enumerate(marks):
    if len(student_marks) > len(columns):
        raise ValueError(f'Marks error!\nLine index: {lesson_index}\n{student_marks}')

In [ ]:
for row_index, student in enumerate(students):
    student_id = student["id"]
    student_name = student["name"]
    student_marks_to_put = marks[row_index]
    
    print(student_name)
    print(student_id)
    
    for lesson_index, mark_to_put in enumerate(student_marks_to_put):
        print(columns[lesson_index]['title'])
        
        lesson_id = columns[lesson_index]['value']
        mark_current = get_mark(student, lesson_id)
        
        if mark_to_put == " ":
            mark_to_put = ""
        if mark_to_put == "y":
            mark_to_put = "н"
        if mark_current == "\u041d":
            mark_current = "н"
        
        if mark_current != mark_to_put:
            put_mark(lesson_id, student_id, mark_to_put)
        
    print()
    